In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

#  Autogluon 기본 사용법
* 1. 데이터 로딩
* 2. 타겟변수 지정
* 3. TabularPredictor 설정. 타겟변수, 모델 성능 지표
* 4. 훈련(데이터, 제한시간설정, 분석사진설정 지정)
* 5. 데이터에서 일부 데이터를 테스트 데이터로 추출
* 6. 분석이 끝난 모델로 테스트 데이터에서 추론 .predict()
* 7. 평가

# 1. 데이터 로딩

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ADsP/main/salary2.csv")
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train_data, test_data = train_test_split(data, stratify=data['class'], test_size=0.4, random_state=10)

In [5]:
train_data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
12689,49,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,35,United-States,<=50K
30011,45,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,44,United-States,>50K
28454,55,Local-gov,HS-grad,9,Married-civ-spouse,Prof-specialty,Other-relative,White,Female,0,2246,40,United-States,>50K
17394,69,Private,HS-grad,9,Married-civ-spouse,Sales,Wife,White,Female,0,0,40,United-States,<=50K
31597,21,Private,Some-college,10,Never-married,Machine-op-inspct,Own-child,White,Female,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20424,31,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
28686,18,NaN,HS-grad,9,Never-married,NaN,Own-child,White,Female,0,0,40,United-States,<=50K
30381,28,Private,Some-college,10,Never-married,Transport-moving,Not-in-family,White,Male,0,0,50,United-States,<=50K
48693,61,Private,Some-college,10,Divorced,Other-service,Not-in-family,Black,Male,0,0,40,United-States,<=50K


In [6]:
test_data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
22332,56,NaN,HS-grad,9,Divorced,NaN,Not-in-family,White,Male,0,0,10,United-States,<=50K
33515,57,Private,HS-grad,9,Divorced,Adm-clerical,Unmarried,White,Female,0,0,42,United-States,<=50K
39475,37,State-gov,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,United-States,<=50K
11140,19,Private,Some-college,10,Never-married,Other-service,Not-in-family,White,Female,0,0,20,United-States,<=50K
39998,41,State-gov,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43439,21,Private,Some-college,10,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,45,United-States,<=50K
3527,28,Self-emp-inc,Bachelors,13,Never-married,Exec-managerial,Own-child,White,Male,0,0,60,United-States,<=50K
46661,27,Private,HS-grad,9,Separated,Machine-op-inspct,Not-in-family,White,Male,0,0,43,United-States,<=50K
28427,38,Private,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,<=50K


# 2. 타겟변수 지정

In [8]:
target_column = "class"

# 3. 제한시간, 검정지표(평가지표, accuracy, rmse, roc_auc) 지정

In [11]:
# 초 단위로 제한시간 지정 300초는 5분,
time_limit = 300
# 성능지표(accuracy, f1, recall, precision, roc_auc_score, mse,rmse,r2)
metric = 'accuracy'

# 4. 모델정의 TabularPredictor()

In [12]:
model = TabularPredictor(label=target_column, eval_metric=metric)

No path specified. Models will be saved in: "AutogluonModels/ag-20241017_025612"


# 모델 훈련

In [13]:
model.fit(train_data, time_limit=time_limit, presets='medium_quality')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
Memory Avail:       13.68 GB / 15.31 GB (89.3%)
Disk Space Avail:   81.11 GB / 237.85 GB (34.1%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels/ag-20241017_025612"
Train Data Rows:    29305
Train Data Columns: 13
Label Column:       class
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       binary
Preprocessing d

	Ran out of time, stopping training early. (Stopping on epoch 1)
/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/autogluon/tabular/models/tabular_nn/torch/tabular_nn_torch.py:411: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an iss

# 6. 생성된 모델에 테스트 데이터 넣어 예측하기

In [14]:
pred = model.predict(test_data)

# 7. 모델 성능 평가하기

In [15]:
result = model.evaluate(test_data)
result_df = pd.DataFrame([result], index =[0])
# 여러 모델 성능 비교
leader_board = model.leaderboard(test_data)
# 중요 변수 출력
feature_importance = model.feature_importance(test_data)
best_model_name = model.model_best
# best 모델 로딩
best_model = model._trainer.load_model(best_model_name)
best_model_params = best_model.params

Computing feature importance via permutation shuffling for 13 features using 5000 rows with 5 shuffle sets...
	166.92s	= Expected runtime (33.38s per shuffle set)
	150.96s	= Actual runtime (Completed 5 of 5 shuffle sets)


# 8. 결과 출력

In [18]:
print("="*20, "result_df", "="*20)
display(result_df)
print()
print("="*20, "leader_board", "="*20)
display(leader_board)
print()
print("="*20, "feature_importance", "="*20)
display(feature_importance)
print()
print("="*20, "best_model_name, params", "="*20)
display("best_model_name: ", best_model_name , "params: ", best_model_params)
print()

==================== result_df ====================


,accuracy,balanced_accuracy,mcc,roc_auc,f1,precision,recall
0,0.875058,0.797504,0.639324,0.925742,0.71306,0.791493,0.64877



==================== leader_board ====================


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.875416,0.8772,accuracy,0.072437,0.010256,0.946533,0.072437,0.010256,0.946533,1,True,11
1,WeightedEnsemble_L2,0.875058,0.8784,accuracy,8.747597,1.186119,244.403419,0.005327,0.000699,0.121793,2,True,14
2,LightGBM,0.872601,0.8764,accuracy,0.023810,0.011164,0.482496,0.023810,0.011164,0.482496,1,True,4
3,CatBoost,0.872550,0.8756,accuracy,0.040281,0.007470,5.407236,0.040281,0.007470,5.407236,1,True,7
4,LightGBMLarge,0.870093,0.8740,accuracy,0.027555,0.005081,0.836579,0.027555,0.005081,0.836579,1,True,13
5,LightGBMXT,0.867226,0.8700,accuracy,0.041468,0.006686,1.213520,0.041468,0.006686,1.213520,1,True,3
6,NeuralNetFastAI,0.856733,0.8624,accuracy,8.629552,1.167693,237.927857,8.629552,1.167693,237.927857,1,True,10
7,RandomForestGini,0.855044,0.8504,accuracy,1.091653,0.124002,1.396219,1.091653,0.124002,1.396219,1,True,5
8,RandomForestEntr,0.854891,0.8512,accuracy,1.084640,0.114926,1.313387,1.084640,0.114926,1.313387,1,True,6
9,ExtraTreesEntr,0.849670,0.8432,accuracy,1.510243,0.125880,1.367323,1.510243,0.125880,1.367323,1,True,9



==================== feature_importance ====================


...


==================== best_model_name, params ====================


'best_model_name: '

'WeightedEnsemble_L2'

'params: '

{'use_orig_features': False,
 'max_base_models': 25,
 'max_base_models_per_type': 5,
 'save_bag_folds': True}

In [26]:
!pip install gradio

  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 51.9 MB/s eta 0:00:0000:0100:01
Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.4 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 47.7 MB/s eta 0:00:0000:01
Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
Using cached ffmpy-0.4.0-py3-none-any.whl (5.8 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.1
    Uninstalling MarkupSafe-3.0.1:
      Successfully uninstalled MarkupSafe-3.0.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17


# 함수화하고 분석 간단히 하기

In [24]:
import gradio as gr
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

def automl(data, target, time=300, metric=None):
    
    
    
    if metric in ("accuracy", "roc_auc", "recall", "precision", "f1"):
        train_data, test_data = train_test_split(data, stratify=data[target], test_size=0.4, random_state=10)
    else:
        train_data, test_data = train_test_split(data, test_size=0.4, random_state=10)
    

    

    model = TabularPredictor(label=target, eval_metric=metric)
    model.fit(train_data, time_limit=time_limit, presets='medium_quality')
    pred = model.predict(test_data)

    result = model.evaluate(test_data)
    result_df = pd.DataFrame([result], index =[0])
    # 여러 모델 성능 비교
    leader_board = model.leaderboard(test_data)
    # 중요 변수 출력
    feature_importance = model.feature_importance(test_data)
    best_model_name = model.model_best
    # best 모델 로딩
    best_model = model._trainer.load_model(best_model_name)
    best_model_params = best_model.params

    print("="*20, "result_df", "="*20)
    display(result_df)
    print()
    print("="*20, "leader_board", "="*20)
    display(leader_board)
    print()
    print("="*20, "feature_importance", "="*20)
    display(feature_importance)
    print()
    print("="*20, "best_model_name, params", "="*20)
    display("best_model_name: ", best_model_name , "params: ", best_model_params)
    print()
    return best_model, result_df, leader_board, feature_importance

In [21]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/Taitanic_train.csv")
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [25]:
automl(data, 'Survived')

No path specified. Models will be saved in: "AutogluonModels/ag-20241017_033748"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
Memory Avail:       12.91 GB / 15.31 GB (84.3%)
Disk Space Avail:   79.83 GB / 237.85 GB (33.6%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels/ag-20241017_033748"
Train Data Rows:    534
Train Data Columns: 11
Label Column:       Survived
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 

/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/autogluon/tabular/models/tabular_nn/torch/tabular_nn_torch.py:411: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

==================== result_df ====================


,accuracy,balanced_accuracy,mcc,roc_auc,f1,precision,recall
0,0.817927,0.778828,0.593756,0.856475,0.716157,0.811881,0.640625



==================== leader_board ====================


...

'best_model_name: '

'WeightedEnsemble_L2'

'params: '

{'use_orig_features': False,
 'max_base_models': 25,
 'max_base_models_per_type': 5,
 'save_bag_folds': True}

(<autogluon.core.models.ensemble.weighted_ensemble_model.WeightedEnsembleModel at 0x7faf27695000>,
    accuracy  balanced_accuracy       mcc  ...        f1  precision    recall
 0  0.817927           0.778828  0.593756  ...  0.716157   0.811881  0.640625
 
 [1 rows x 7 columns],
                   model  score_test  ...  can_infer fit_order
 0              LightGBM    0.817927  ...       True         4
 1   WeightedEnsemble_L2    0.817927  ...       True        14
 2       NeuralNetFastAI    0.815126  ...       True        10
 3        NeuralNetTorch    0.812325  ...       True        12
 4         LightGBMLarge    0.809524  ...       True        13
 5               XGBoost    0.806723  ...       True        11
 6      RandomForestEntr    0.806723  ...       True         6
 7              CatBoost    0.803922  ...       True         7
 8      RandomForestGini    0.803922  ...       True         5
 9        ExtraTreesEntr    0.801120  ...       True         9
 10           LightGBMXT   